# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [27]:
from openai import OpenAI
client = OpenAI()


In [28]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条穿越田野的木质栈道。栈道两侧是郁郁葱葱的绿色草地和一些低矮的灌木。图像的背景是一片开阔的天空，布满了绵延的白云。光线明亮且自然，突出了这个景象的宁静与自然美。整个场景传递出一种平和与远离城市喧嚣的感觉，是一个引人入胜的自然景观。这样的栈道通常用于观赏自然风景，同时保护脆弱的地表不受游客行走的伤害。', role='assistant', function_call=None, tool_calls=None))


In [29]:
response.choices[0].message.content

'这幅图展示了一条穿越田野的木质栈道。栈道两侧是郁郁葱葱的绿色草地和一些低矮的灌木。图像的背景是一片开阔的天空，布满了绵延的白云。光线明亮且自然，突出了这个景象的宁静与自然美。整个场景传递出一种平和与远离城市喧嚣的感觉，是一个引人入胜的自然景观。这样的栈道通常用于观赏自然风景，同时保护脆弱的地表不受游客行走的伤害。'

### 封装成一个函数 query_image_description

In [30]:
def query_image_description(url, prompt="介绍下这幅图?"):
    # client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [31]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一个幽默图，通过对比展示了两种极端的狗狗形象。左侧的狗被描绘成具有非常夸张的肌肉和人类体形的生物，标签为“16岁的我——工作后的我”，上面有几句话描述它强壮、主动、自信以及具有领导者品质。

右侧的狗则看起来更为普通、柔弱，被标识为体现了工作后的现状：经历世事的辛酸，感到身心疲惫且略显沮丧。

这个幽默图显然是用来表达人们对青春时期的理想和工作后实际状态的戏剧化对比，通过幽默的形式表达了成年后面临的压力和变化。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [32]:
from openai import OpenAI
import base64
import requests
import json

# client = OpenAI()  # 初始化 OpenAI 客户端


def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [33]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图显示了1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）比较。横轴代表年份，纵轴代表GDP，单位是万亿美元。

从图中我们可以看到以下几点：
1. 美国的GDP在这40年间持续增长，领先于其他三国。
2. 中国的GDP增长非常迅速，尤其是从2000年之后，其增长速度显著加快，到大约2010年追上并超过日本。
3. 日本的GDP在1990年代早期达到高峰后，增长放缓，之后相对平稳。
4. 德国的GDP增长相对稳定，但增速较慢，始终保持在较低的水平。

通过这个图可以直观地看出各国经济发展的不同阶段和速度，特别是中国的快速崛起以及经济全球化进程中不同国家的变化。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [34]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图是一本笔记本上的手写笔记，内容跟自然语言处理（NLP）有关。笔记介绍了几种用于优化和调整预训练语言模型的方法，特别是在使用较小的模型时。具体内容包括：

1. **Prompt Tuning（提示调整）**: 这是一种通过修改输入提示（输入前几个单词或词组）来调整模型的输出结果的技术。笔记中提到这通常用于较小的模型。

2. **Prefix Tuning（前缀调整）**: 在这种技术中，将特定的前缀词向量加入到模型的输入中，以调整模型在处理输入数据时的行为。

3. **LoRA（Low-Rank Adaptation，低秩适配）**: 通过修改模型权重的低秩矩阵来实现高效的模型调整，特别是在参数数量庞大的模型中，这种方法可以有效减少所需调整的参数数目。

笔记还涉及了Q-LoRA 和模型大小优化的一些数据，比如 LLaMA-653G 模型和 QLoRA-48GB。

总的来说，这些笔记显示了一些先进的技术，用于提升机器学习模型在特定任务上的表现，尤其是在资源有限的情况下。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [35]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图是一本笔记本上的手写笔记，内容跟自然语言处理（NLP）有关。笔记介绍了几种用于优化和调整预训练语言模型的方法，特别是在使用较小的模型时。具体内容包括：

1. **Prompt Tuning（提示调整）**: 这是一种通过修改输入提示（输入前几个单词或词组）来调整模型的输出结果的技术。笔记中提到这通常用于较小的模型。

2. **Prefix Tuning（前缀调整）**: 在这种技术中，将特定的前缀词向量加入到模型的输入中，以调整模型在处理输入数据时的行为。

3. **LoRA（Low-Rank Adaptation，低秩适配）**: 通过修改模型权重的低秩矩阵来实现高效的模型调整，特别是在参数数量庞大的模型中，这种方法可以有效减少所需调整的参数数目。

笔记还涉及了Q-LoRA 和模型大小优化的一些数据，比如 LLaMA-653G 模型和 QLoRA-48GB。

总的来说，这些笔记显示了一些先进的技术，用于提升机器学习模型在特定任务上的表现，尤其是在资源有限的情况下。

![](./images/handwriting_1.jpg)

In [36]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一份笔记，内容主要围绕机器学习和自然语言处理特别是在变换模型（Transformers）的训练和调整方法。笔记中提及了多种技术和概念，其中包括：

1.  **PEFT/SOTA** 和 **PBFT Methods**：提到了最先进（State of the Art，SOTA）的方法和可能指的是某种特定于预训练模型的调整技术。
2.  **Prompt Tuning**：提到了利用提示进行模型微调的多种方法，如 Adapter、Prefix、和 Prompt tuning。这些都是近年来在处理预训练的大型语言模型（如GPT、BERT等）时常用的技术。
3.  **硬提示（Hard Prompts） 和 模板提示 (Template-based Prompts)**：这可能是关于如何设计模型输入以引导模型生成特定类型输出的方法。
4.  **Instruction Tuning**：涉及到指令性调整，即直接通过指令来优化模型的行为。
5.  **模型名称**：像是LLaMA、ChatGLM、GPT-XL、RHF等，这些可能是指不同的语言模型或者其变种。
6.  **LoRA, GLoRA, AdLoRA**：这些可能是不同的技术或方法，用于增强模型在特定任务上的表现或效率。
7.  **MAM Adapters** 和 插入型形态（Insertion Form）等：讨论了模型的结构性调整和适配器层的应用。

笔记右侧部分的内容被部分涂黑，显示作者可能在记录或总结某个具体的知识点时对内容进行了部分删除或修正。这份笔记反映了作者在自然语言处理领域学习和研究的深度，特别是在模型调整方法上的多样化和深入探讨。

![](./images/出行计划.jpg)

In [40]:

content = query_base64_image_description("./images/出行计划.jpg", prompt="识别一下图中的文字，不需要你概括内容，只需识别")

print(content)

图中的文字如下：

行程表（7月21、22、23、24）。

Day 1: 7月21日起程。

① 11:05 — 13:30 纽约至费城（2小时35分钟） （MU2877，中转飞机）

② 从机场在费城市环市内游览：

- 钟楼游览（约20:00达，终

- 从费城前往费城港一块拥有美丽的自然景色及动物园）

   （重点注意：途径曼哈顿地区约12分钟，咖啡店）

   - -

③ 38.1业，行程大约37分钟。在动物园前玩耍，吃饭时间，中间稍事。

   入游园。

- 吃晚饭，SI阶段（勿忘带太阳眼镜）款50分鐘。

   （76）

  秘（素菜放凉6花味）

   （务务期间多留些美国花园总23分钟，3小时。

④ 各种动物包括旺节影14分钟，班吃。

⑤ 表演结束分别在第二场15:10。

⑥ 所有人在事务事务事务事务。

   - 清点

△ 睡一晚；早餐吃得！伦丹、不带撒瑞米枕头与枕头

   1）床垫短时

   2）枕头接触

   3）福网紧缚

   4）枕头表达

   5）床单短缺

   6）福网调整

△ 第二次：

   入场（1次）8:00—17:30；夜市7:30—20:30（形容询问2处）

   大即空摆，先陷睡口一伤，花费近口说话行过，陆时

   罗罗罗罗罗罗罗罗罗罗罗罗罗罗一单向

   已有超600字的清单表。


# 2
整合 query_base64_image_description 函数和 Markdown

In [41]:
from IPython.display import display, Markdown

content = query_base64_image_description("./images/出行计划.jpg", prompt="识别一下图中的文字，不需要你概括内容，只需识别,尽量以markdown方式展示")
display(Markdown(content))

```markdown
待办事项（7月21、22、23、24）

Day 1：7月21日星期日
① 11:05 — 13:30 空闲时间（2小时35分钟）
   （MU2877，中午飞机）
② 吃饭及早关营业厅 13:20前、睡
   从家起床后来单位一体检及交接旅程及交海关行李（根表交到交税12/6, 大厅）
   （市表加交到交税37年表, 住街五层楼时换、快受街, 中山路。
    生费区、S1及第名部部加上而（7记），未办记。
    科（党表初记6记）
    （务表初记已换记）在23时, 3分钟。
    会检计书到生可能14:50分，返线。
③ 文曲星查验车间 15:00-16:30。
④ 没有良好天气, 谈心发声生预备。
⑤ 时间表：思考！并不仅耿耿不息（城市院）已预定批改
    1) 思考良声　　2) 适时素质　　3) 科学知识
    4) 思考世界　　5) 思考质量　　6) 品纯素质
△ 早上注意：回家（1趟 8:00-17:30; 东方 7:30-20:30）世康闭始24款
   大明之前, 无康康一项, 区委开口回家谈授, 山势明察星看东参厂建, 山势就系屋翻阳台的一角窗。
   2才600毫升有所需。
```


## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。